# Making Segmentation

In [1]:
# === Radiology-Style Clinical Summary Table: NSCLC Dataset with Staging and Nodule Metrics ===
import pandas as pd
import numpy as np
from tabulate import tabulate
import os
import json
import numpy as np
import pandas as pd
import SimpleITK as sitk
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
from scipy.ndimage import label, binary_dilation,binary_erosion
import SimpleITK as sitk
import numpy as np
import scipy.ndimage as ndi

# Load NIfTI images using SimpleITK
def load_nifti(file_path):
    img   = sitk.ReadImage(file_path)
    array = sitk.GetArrayFromImage(img)
    return img, array

# Save NIfTI images using SimpleITK
def save_nifti(array, reference_img, output_path):
    updated_img = sitk.GetImageFromArray(array)
    updated_img.CopyInformation(reference_img)  # Keep original metadata
    sitk.WriteImage(updated_img, output_path)
    
    
    


def get_body_mask(ct_image_path, threshold=-300):
    """
    Generate a binary mask of the body from a CT scan.
    
    Args:
        ct_image (sitk.Image): 3D CT image path
        threshold (int): Hounsfield Unit threshold to separate air/background from body (typical: -300)

    Returns:
        sitk.Image: Binary mask (1 = body, 0 = background)
    """
    ct_image = sitk.ReadImage(ct_image_path)
    # Step 1: Thresholding (e.g., air is usually below -500 HU)
    binary = sitk.BinaryThreshold(ct_image, lowerThreshold=threshold, upperThreshold=3071, insideValue=1, outsideValue=0)

    # Step 2: Remove small air pockets inside the body using connected components
    # Use largest connected component to eliminate isolated noise
    cc = sitk.ConnectedComponent(binary)
    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.Execute(cc)
    
    largest_label = max(stats.GetLabels(), key=lambda l: stats.GetPhysicalSize(l))
    body_mask = sitk.BinaryThreshold(cc, lowerThreshold=largest_label, upperThreshold=largest_label, insideValue=1, outsideValue=0)

    # Step 3: Fill holes inside the body mask (optional, improves quality)
    body_mask = sitk.VotingBinaryHoleFilling(body_mask, radius=[2]*3, majorityThreshold=1)
    body_img = sitk.GetArrayFromImage(body_mask)

    return body_img






#--- Sehmemntation Paths 
# luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.999671985647416666415767161891_0000_trans.nii.gz

Vista3D_segmenation_path   = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/Segmentation_results/Vista3D_auto_eval_LUNA25tr_allseg/'
org_segmenation_path       = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/LUNA25_KNN_2mm_Extend_Seg/'
psedo_Segmentation_path    = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/vista3Dauto_seg_knneX2mm_GTV/'
ct_path                    = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/luna25_images_nifti/'




Nifti_name_List   = [f for f in os.listdir(ct_path) if f.endswith('.nii.gz')]
error             = []

for idx, Nifti in enumerate(Nifti_name_List):
    try:
        segNifti                    = Nifti.split('.nii')[0]+'/'+Nifti.split('.nii')[0]+'_trans.nii.gz'
        pseudo_img, pseudo_array    = load_nifti(Vista3D_segmenation_path+segNifti)
        org_segfile_name            = Nifti.split('.nii')[0]+"_mask.nii.gz"
        org_img,    org_array       = load_nifti(org_segmenation_path+org_segfile_name)
        org_array = binary_erosion(org_array, structure=np.ones((3, 3, 3)))
        pseudo_array[org_array==1]  = 23
        body_array = get_body_mask(ct_path+Nifti, threshold=-300)
        pseudo_array[(body_array != 0) & (pseudo_array == 0)] = 200
        out_put_save_path = psedo_Segmentation_path+Nifti.split('.nii')[0]+'_seg_sh.nii.gz' 
        save_nifti(pseudo_array, pseudo_img, out_put_save_path)
        print('{}|Reading|{}'.format(idx+1,segNifti))
    except:
        print('{}|Reading|{}'.format(idx+1,segNifti))
        pass
        




1|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.336082917684408755057011874628_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.336082917684408755057011874628_0000_trans.nii.gz
2|Reading|luna25_1.2.840.113654.2.55.324639171454099040823837420317635634877_0000/luna25_1.2.840.113654.2.55.324639171454099040823837420317635634877_0000_trans.nii.gz
3|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.266841853879722706638309780920_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.266841853879722706638309780920_0000_trans.nii.gz
4|Reading|luna25_1.2.840.113654.2.55.136328495082999166608130156525127074982_0000/luna25_1.2.840.113654.2.55.136328495082999166608130156525127074982_0000_trans.nii.gz
5|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.113485382374310401425757079548_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.113485382374310401425757079548_0000_trans.nii.gz
6|Reading|luna25_1.2.840.113654.2.55.303452086938156278902787236649672496089_0000/luna25_1.2.840.113654.2.55.30345208693815627890278723

50|Reading|luna25_1.2.840.113654.2.55.265009986341160012726589108259209467223_0000/luna25_1.2.840.113654.2.55.265009986341160012726589108259209467223_0000_trans.nii.gz
51|Reading|luna25_1.2.840.113654.2.55.95777266530018336200609740955121863480_0000/luna25_1.2.840.113654.2.55.95777266530018336200609740955121863480_0000_trans.nii.gz
52|Reading|luna25_1.2.840.113654.2.55.51905868808044674544987759478595651504_0000/luna25_1.2.840.113654.2.55.51905868808044674544987759478595651504_0000_trans.nii.gz
53|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.144597356678732949948307951503_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.144597356678732949948307951503_0000_trans.nii.gz
54|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.580403672217272585768095807881_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.580403672217272585768095807881_0000_trans.nii.gz
55|Reading|luna25_1.2.840.113654.2.55.114036864609995620634310167491799496839_0000/luna25_1.2.840.113654.2.55.1140368646099956206343101674917994

99|Reading|luna25_1.2.840.113654.2.55.173400607642827664862791225595154784969_0000/luna25_1.2.840.113654.2.55.173400607642827664862791225595154784969_0000_trans.nii.gz
100|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.222369185589738792111344729886_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.222369185589738792111344729886_0000_trans.nii.gz
101|Reading|luna25_1.2.840.113654.2.55.36093908484383947151728743809946393001_0000/luna25_1.2.840.113654.2.55.36093908484383947151728743809946393001_0000_trans.nii.gz
102|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.241435461463862772640365635218_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.241435461463862772640365635218_0000_trans.nii.gz
103|Reading|luna25_1.2.840.113654.2.55.92132714321990365519624534928090421522_0000/luna25_1.2.840.113654.2.55.92132714321990365519624534928090421522_0000_trans.nii.gz
104|Reading|luna25_1.2.840.113654.2.55.202329971943245504510619802648529839228_0000/luna25_1.2.840.113654.2.55.20232997194324550451061980264

147|Reading|luna25_1.2.840.113654.2.55.21161027445172429994705257071187713094_0000/luna25_1.2.840.113654.2.55.21161027445172429994705257071187713094_0000_trans.nii.gz
148|Reading|luna25_1.2.840.113654.2.55.31134773124962680314971420470436141970_0000/luna25_1.2.840.113654.2.55.31134773124962680314971420470436141970_0000_trans.nii.gz
149|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.144418570860051742229717895599_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.144418570860051742229717895599_0000_trans.nii.gz
150|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.210299657503988940943277329574_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.210299657503988940943277329574_0000_trans.nii.gz
151|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.174596381655198264267369281694_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.174596381655198264267369281694_0000_trans.nii.gz
152|Reading|luna25_1.2.840.113654.2.55.73348593030555988002263232174103951909_0000/luna25_1.2.840.113654.2.55.7334859303055598800

195|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.577162664505312760102823499916_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.577162664505312760102823499916_0000_trans.nii.gz
196|Reading|luna25_1.2.840.113654.2.55.125640785978179426297394044984562171806_0000/luna25_1.2.840.113654.2.55.125640785978179426297394044984562171806_0000_trans.nii.gz
197|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.339835645798758446691532724163_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.339835645798758446691532724163_0000_trans.nii.gz
198|Reading|luna25_1.2.840.113654.2.55.295762702402940627335780432042531588905_0000/luna25_1.2.840.113654.2.55.295762702402940627335780432042531588905_0000_trans.nii.gz
199|Reading|luna25_1.2.840.113654.2.55.51610868684916323930539877097733839139_0000/luna25_1.2.840.113654.2.55.51610868684916323930539877097733839139_0000_trans.nii.gz
200|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.108408954612534269043593729438_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.1084089

...

3838|Reading|luna25_1.2.840.113654.2.55.285850339727801660418073848851176369986_0000/luna25_1.2.840.113654.2.55.285850339727801660418073848851176369986_0000_trans.nii.gz
3839|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.280944219334737775271816996585_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.280944219334737775271816996585_0000_trans.nii.gz
3840|Reading|luna25_1.2.840.113654.2.55.76097901340351258558564393548301988147_0000/luna25_1.2.840.113654.2.55.76097901340351258558564393548301988147_0000_trans.nii.gz
3841|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.102741113990094214108703437702_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.102741113990094214108703437702_0000_trans.nii.gz
3842|Reading|luna25_1.2.840.113654.2.55.282173032626697583174937703730527979782_0000/luna25_1.2.840.113654.2.55.282173032626697583174937703730527979782_0000_trans.nii.gz
3843|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.630515084985736492348180603421_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.6

3885|Reading|luna25_1.2.840.113654.2.55.227597284934443401306954761113910779761_0000/luna25_1.2.840.113654.2.55.227597284934443401306954761113910779761_0000_trans.nii.gz
3886|Reading|luna25_1.2.840.113654.2.55.67658998447479018042022592870370600614_0000/luna25_1.2.840.113654.2.55.67658998447479018042022592870370600614_0000_trans.nii.gz
3887|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.244343968367592778661145152451_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.244343968367592778661145152451_0000_trans.nii.gz
3888|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.133103585381563066524943821845_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.133103585381563066524943821845_0000_trans.nii.gz
3889|Reading|luna25_1.2.840.113654.2.55.94565769154813686772638714474256379277_0000/luna25_1.2.840.113654.2.55.94565769154813686772638714474256379277_0000_trans.nii.gz
3890|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.114038757122272447573643002129_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.114

3933|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.903255672067278170725007257023_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.903255672067278170725007257023_0000_trans.nii.gz
3934|Reading|luna25_1.2.840.113654.2.55.307041535554763778217582669592778314000_0000/luna25_1.2.840.113654.2.55.307041535554763778217582669592778314000_0000_trans.nii.gz
3935|Reading|luna25_1.2.840.113654.2.55.91413877555784822535477457727475296101_0000/luna25_1.2.840.113654.2.55.91413877555784822535477457727475296101_0000_trans.nii.gz
3936|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.174790752976787283187885122551_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.174790752976787283187885122551_0000_trans.nii.gz
3937|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.233253643922570418503774287719_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.233253643922570418503774287719_0000_trans.nii.gz
3938|Reading|luna25_1.2.840.113654.2.55.265655744316735135276276142427649239630_0000/luna25_1.2.840.113654.2.55.2656557443

3981|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.126771866502337542200291083051_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.126771866502337542200291083051_0000_trans.nii.gz
3982|Reading|luna25_1.2.840.113654.2.55.274994675324176114454465843693448656470_0000/luna25_1.2.840.113654.2.55.274994675324176114454465843693448656470_0000_trans.nii.gz
3983|Reading|luna25_1.2.840.113654.2.55.94757059152311931493258288414653636971_0000/luna25_1.2.840.113654.2.55.94757059152311931493258288414653636971_0000_trans.nii.gz
3984|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.177451940373114216490377003846_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.177451940373114216490377003846_0000_trans.nii.gz
3985|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.304371071883668569319808850659_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.304371071883668569319808850659_0000_trans.nii.gz
3986|Reading|luna25_1.2.840.113654.2.55.2726407576546184014343417707155943942_0000/luna25_1.2.840.113654.2.55.272640757654

4029|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.163991350894128718582324243933_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.163991350894128718582324243933_0000_trans.nii.gz
4030|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.258447067697011452485679843273_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.258447067697011452485679843273_0000_trans.nii.gz
4031|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.363798956828105313556489041345_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.363798956828105313556489041345_0000_trans.nii.gz
4032|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.130752257197167813590681525870_0000/luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.130752257197167813590681525870_0000_trans.nii.gz
4033|Reading|luna25_1.2.840.113654.2.55.135312532800636306422912105667791824180_0000/luna25_1.2.840.113654.2.55.135312532800636306422912105667791824180_0000_trans.nii.gz
4034|Reading|luna25_1.2.840.113654.2.55.109935711018211239275977773631810109196_0000/luna25_1.2.840.113654.2.5

# Benign and Malignant

In [6]:
import pandas as pd
import json
import os
# === Radiology-Style Clinical Summary Table: NSCLC Dataset with Staging and Nodule Metrics ===
import pandas as pd
import numpy as np
from tabulate import tabulate
import os
import json
import numpy as np
import pandas as pd
import SimpleITK as sitk
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
from scipy.ndimage import label, binary_dilation,binary_erosion
import SimpleITK as sitk
import numpy as np
import scipy.ndimage as ndi

# Load NIfTI images using SimpleITK
def load_nifti(file_path):
    img   = sitk.ReadImage(file_path)
    array = sitk.GetArrayFromImage(img)
    return img, array

# Save NIfTI images using SimpleITK
def save_nifti(array, reference_img, output_path):
    updated_img = sitk.GetImageFromArray(array)
    updated_img.CopyInformation(reference_img)  # Keep original metadata
    sitk.WriteImage(updated_img, output_path)


LUNA25_df = pd.read_csv('/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/LUNA25_Public_Training_Development_Data_AnnotationID_nifti_splitV1.csv')
LUNA25_df_uni = LUNA25_df.drop_duplicates(subset='SeriesInstanceUID',keep='first')
LUNA25_df_uni = LUNA25_df_uni.reset_index(drop=True)

LUNA25_Resampled_spacing              = pd.read_csv('/data/usr/ft42/CVIT_XAI/LungRADS_Modeling/CLARITY/MAIASI_DatasetGen/metadata_scr/LUNA25_512xy_256z_771p25m_SpaingSize_info.csv')
LUNA25_Resampled_spacing['SeriesInstanceUID'] = LUNA25_Resampled_spacing['filename'].str.split('_').str[1]
reject_ct                             = LUNA25_Resampled_spacing[LUNA25_Resampled_spacing['x_size']!=512]['SeriesInstanceUID'].tolist() +  LUNA25_Resampled_spacing[LUNA25_Resampled_spacing['y_size']!=512]['SeriesInstanceUID'].tolist() + LUNA25_Resampled_spacing[LUNA25_Resampled_spacing['z_size']!=256]['SeriesInstanceUID'].tolist()
LUNA25_df_uni                  = LUNA25_df_uni[~LUNA25_df_uni['SeriesInstanceUID'].isin(reject_ct)].reset_index(drop=True)
LUNA25_df_uni



Vista3D_segmenation_path   = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/Segmentation_results/Vista3D_auto_eval_LUNA25tr_allseg/'
org_segmenation_path       = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/LUNA25_KNN_2mm_Extend_Seg/'
psedo_Segmentation_path    = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/vista3Dauto_seg_knneX2mm_GTV/'
ct_path                    = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/luna25_images_nifti/'


psedo_seg_benignMaliganat_path = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/vista3Dauto_seg_knneX2mm_GTV_BenignMalignant/'

Nifti_name_List   = [f for f in os.listdir(psedo_Segmentation_path) if f.endswith('.nii.gz')]

Nifti_name_List_df = pd.DataFrame(list(zip(Nifti_name_List)),columns=['pseudo_seg'])
Nifti_name_List_df['SeriesInstanceUID'] = Nifti_name_List_df['pseudo_seg'].str.split('_').str[1]
Nifti_name_List_df = pd.merge(Nifti_name_List_df,LUNA25_df_uni[['SeriesInstanceUID','label']],on='SeriesInstanceUID',how='right')
Nifti_name_List_df

error             = []

for idx, Nifti in enumerate(Nifti_name_List_df['pseudo_seg'].tolist()):
    try:
        pseudo_img, pseudo_array    = load_nifti(psedo_Segmentation_path+Nifti)
        
        if Nifti_name_List_df['label'][idx] == 0:
            pseudo_array[pseudo_array==23]  = 2
        if Nifti_name_List_df['label'][idx] == 1:
            pseudo_array[pseudo_array==23]  = 16

        out_put_save_path = psedo_seg_benignMaliganat_path+Nifti
        save_nifti(pseudo_array, pseudo_img, out_put_save_path)
        print('{}|Reading|{}'.format(idx+1,Nifti))
    except:
        print('{}|Reading|{}'.format(idx+1,Nift))
        pass

1|Reading|luna25_1.2.840.113654.2.55.75732636705528098841640044100937337510_0000_seg_sh.nii.gz
2|Reading|luna25_1.2.840.113654.2.55.170326365301615965510940077306443749115_0000_seg_sh.nii.gz
3|Reading|luna25_1.2.840.113654.2.55.252394161569626563347616024146752427830_0000_seg_sh.nii.gz
4|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.159197566713017635281317443032_0000_seg_sh.nii.gz
5|Reading|luna25_1.2.840.113654.2.55.281830784862850116853977282407476115680_0000_seg_sh.nii.gz
6|Reading|luna25_1.2.840.113654.2.55.203402510321043650335833757027834687105_0000_seg_sh.nii.gz
7|Reading|luna25_1.2.840.113654.2.55.141751942912405516148110040638470627452_0000_seg_sh.nii.gz
8|Reading|luna25_1.2.840.113654.2.55.66526943704704440656652321658161947738_0000_seg_sh.nii.gz
9|Reading|luna25_1.2.840.113654.2.55.15671098524735946274796626542954424985_0000_seg_sh.nii.gz
10|Reading|luna25_1.2.840.113654.2.55.294423248031318124872302490007395358486_0000_seg_sh.nii.gz
11|Reading|luna25_1.2.840.113654.2.55

85|Reading|luna25_1.2.840.113654.2.55.263416585793327985204627316526894166371_0000_seg_sh.nii.gz
86|Reading|luna25_1.2.840.113654.2.55.138912617889370319628771318657755494354_0000_seg_sh.nii.gz
87|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.195185449252908969413846478145_0000_seg_sh.nii.gz
88|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.292687058865736938413969610961_0000_seg_sh.nii.gz
89|Reading|luna25_1.2.840.113654.2.55.112986426358634422297285376913177317507_0000_seg_sh.nii.gz
90|Reading|luna25_1.2.840.113654.2.55.87796000685588733324625280071586129280_0000_seg_sh.nii.gz
91|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.503793032347775813042792543782_0000_seg_sh.nii.gz
92|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.329417386081599044461362638491_0000_seg_sh.nii.gz
93|Reading|luna25_1.2.840.113654.2.55.151266835241366964662127512594686819186_0000_seg_sh.nii.gz
94|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.272205026898005258368394989404_0000_seg_sh.nii.gz
95|Rea

168|Reading|luna25_1.2.840.113654.2.55.288762201003595736092349141868796446880_0000_seg_sh.nii.gz
169|Reading|luna25_1.2.840.113654.2.55.98107972720003998892122802263137643444_0000_seg_sh.nii.gz
170|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.120265265754391550520121365421_0000_seg_sh.nii.gz
171|Reading|luna25_1.2.840.113654.2.55.337680951516292532993550974033181854899_0000_seg_sh.nii.gz
172|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.168036211510312543747050239345_0000_seg_sh.nii.gz
173|Reading|luna25_1.2.840.113654.2.55.46608830063142635336535908776842833622_0000_seg_sh.nii.gz
174|Reading|luna25_1.2.840.113654.2.55.309367576722439202240311013018082137778_0000_seg_sh.nii.gz
175|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.131789187696638200912229583752_0000_seg_sh.nii.gz
176|Reading|luna25_1.2.840.113654.2.55.192765305213821544184714012377307952051_0000_seg_sh.nii.gz
177|Reading|luna25_1.2.840.113654.2.55.264302944167758476519315132013963453758_0000_seg_sh.nii.gz
178|Rea

251|Reading|luna25_1.2.840.113654.2.55.121225028393431002977301089904117534579_0000_seg_sh.nii.gz
252|Reading|luna25_1.2.840.113654.2.55.134499607175505455351765902761934752797_0000_seg_sh.nii.gz
253|Reading|luna25_1.2.840.113654.2.55.280863748116675799728722524654577950248_0000_seg_sh.nii.gz
254|Reading|luna25_1.2.840.113654.2.55.288754000579084188132699391831194885777_0000_seg_sh.nii.gz
255|Reading|luna25_1.2.840.113654.2.55.288638815680111681127103675558531201499_0000_seg_sh.nii.gz
256|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.126936655541485817046843906127_0000_seg_sh.nii.gz
257|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.645097024652158306745911876785_0000_seg_sh.nii.gz
258|Reading|luna25_1.2.840.113654.2.55.224723978404284643188100358086716031019_0000_seg_sh.nii.gz
259|Reading|luna25_1.2.840.113654.2.55.325510501408637201088971035649745412970_0000_seg_sh.nii.gz
260|Reading|luna25_1.2.840.113654.2.55.103763961290875647613399736918730504649_0000_seg_sh.nii.gz
261|Readin

334|Reading|luna25_1.2.840.113654.2.55.292723223320952085527880999997349434890_0000_seg_sh.nii.gz
335|Reading|luna25_1.2.840.113654.2.55.241404684039532801738207950123026659745_0000_seg_sh.nii.gz
336|Reading|luna25_1.2.840.113654.2.55.271534868697895430368453905103321816375_0000_seg_sh.nii.gz
337|Reading|luna25_1.2.840.113654.2.55.308587354974293512959220245321981504107_0000_seg_sh.nii.gz
338|Reading|luna25_1.2.840.113654.2.55.238519687129394139924938635204494068809_0000_seg_sh.nii.gz
339|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.261837924061910977680979201624_0000_seg_sh.nii.gz
340|Reading|luna25_1.2.840.113654.2.55.26059161946299599599170789033398681899_0000_seg_sh.nii.gz
341|Reading|luna25_1.2.840.113654.2.55.284190134409320838451555918854936176980_0000_seg_sh.nii.gz
342|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.314870249527289849830192640001_0000_seg_sh.nii.gz
343|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.143694214772485075472648994040_0000_seg_sh.nii.gz
344|Re

...

3459|Reading|luna25_1.2.840.113654.2.55.15295906577819495609767444377825579411_0000_seg_sh.nii.gz
3460|Reading|luna25_1.2.840.113654.2.55.164181492927394748724133720573330012268_0000_seg_sh.nii.gz
3461|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.115359602665502706444351680406_0000_seg_sh.nii.gz
3462|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.145745567373257852212520912375_0000_seg_sh.nii.gz
3463|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.299104902542759848026265893002_0000_seg_sh.nii.gz
3464|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.333198654235914621742709897900_0000_seg_sh.nii.gz
3465|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.412776161107202222430016896917_0000_seg_sh.nii.gz
3466|Reading|luna25_1.2.840.113654.2.55.78958542850344222630066635318925031987_0000_seg_sh.nii.gz
3467|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.293157896808411809776006065082_0000_seg_sh.nii.gz
3468|Reading|luna25_1.2.840.113654.2.55.174639415508620935145028282277042780072_0

3541|Reading|luna25_1.2.840.113654.2.55.295503707065946314974570533833346343329_0000_seg_sh.nii.gz
3542|Reading|luna25_1.2.840.113654.2.55.255854454983647263209158788400906020793_0000_seg_sh.nii.gz
3543|Reading|luna25_1.2.840.113654.2.55.238342329224122320452539295174558074956_0000_seg_sh.nii.gz
3544|Reading|luna25_1.2.840.113654.2.55.91413877555784822535477457727475296101_0000_seg_sh.nii.gz
3545|Reading|luna25_1.2.840.113654.2.55.205381956539561500879753468394682821578_0000_seg_sh.nii.gz
3546|Reading|luna25_1.2.840.113654.2.55.269524571988971111010182722799497224934_0000_seg_sh.nii.gz
3547|Reading|luna25_1.2.840.113654.2.55.305185589732661725854978371096078399298_0000_seg_sh.nii.gz
3548|Reading|luna25_1.2.840.113654.2.55.141600997935022662288067899669081613283_0000_seg_sh.nii.gz
3549|Reading|luna25_1.2.840.113654.2.55.255298045766679977296292618717619397173_0000_seg_sh.nii.gz
3550|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.119622826982756819393116273236_0000_seg_sh.nii.gz
3551|R

3623|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.995392857716432377449469551081_0000_seg_sh.nii.gz
3624|Reading|luna25_1.2.840.113654.2.55.209819418255569371918529562072819807899_0000_seg_sh.nii.gz
3625|Reading|luna25_1.2.840.113654.2.55.99854803716576675485484799881244990106_0000_seg_sh.nii.gz
3626|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.339307759052637678887373020763_0000_seg_sh.nii.gz
3627|Reading|luna25_1.2.840.113654.2.55.331338434268361798401068174683527841513_0000_seg_sh.nii.gz
3628|Reading|luna25_1.2.840.113654.2.55.183125364895286732850640076263563762479_0000_seg_sh.nii.gz
3629|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.200709384851500584708415897699_0000_seg_sh.nii.gz
3630|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.194845886139071193586980460333_0000_seg_sh.nii.gz
3631|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.291214500058745272448375536990_0000_seg_sh.nii.gz
3632|Reading|luna25_1.2.840.113654.2.55.313830510988592728122388321110697851410_0000_

3705|Reading|luna25_1.2.840.113654.2.55.79867373684197691640881104551435728356_0000_seg_sh.nii.gz
3706|Reading|luna25_1.2.840.113654.2.55.33845633917773133455135930921934655042_0000_seg_sh.nii.gz
3707|Reading|luna25_1.2.840.113654.2.55.87648124097080323122155921680314277847_0000_seg_sh.nii.gz
3708|Reading|luna25_1.2.840.113654.2.55.213329280600207057483868632992229543210_0000_seg_sh.nii.gz
3709|Reading|luna25_1.2.840.113654.2.55.262111473648878566009147292751807028710_0000_seg_sh.nii.gz
3710|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.116942401455626352041703057022_0000_seg_sh.nii.gz
3711|Reading|luna25_1.2.840.113654.2.55.150288324502578083538155600497569671887_0000_seg_sh.nii.gz
3712|Reading|luna25_1.2.840.113654.2.55.75283364522869321785680531299060999736_0000_seg_sh.nii.gz
3713|Reading|luna25_1.2.840.113654.2.55.123098202117641071957698394124500839347_0000_seg_sh.nii.gz
3714|Reading|luna25_1.2.840.113654.2.55.257183259885605042606683226212366935776_0000_seg_sh.nii.gz
3715|Read

3787|Reading|luna25_1.2.840.113654.2.55.93471335636297621128732688640257890204_0000_seg_sh.nii.gz
3788|Reading|luna25_1.2.840.113654.2.55.143717709118838917409716272145088944485_0000_seg_sh.nii.gz
3789|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.533873559327360856749149509577_0000_seg_sh.nii.gz
3790|Reading|luna25_1.2.840.113654.2.55.194026076841803033732108984206340803715_0000_seg_sh.nii.gz
3791|Reading|luna25_1.2.840.113654.2.55.63719678425490222858761645283404194532_0000_seg_sh.nii.gz
3792|Reading|luna25_1.3.6.1.4.1.14519.5.2.1.7009.9004.148086071885379118597541960873_0000_seg_sh.nii.gz
